In [1]:
%load_ext autoreload
%autoreload 2

# GridDataset

Nata represents grids by the `GridDataset` container which can be imported by running

In [2]:
import sys

import numpy as np

from nata.containers import GridDataset
from nata.containers.GridDataset import GridDatasetAxes
from nata.axes import Axis

The next sections discuss the API of `GridDataset`, how to create a grid using the `GridDataset` class, and the basic properties of those objects. In the following, any reference to a grid refers to a instance while `GridDataset` represent the class definition.

## Note on grids

Nata has two ways to create a grid, using an array-like object or from an path as the basis. The creation of a grid from an array is a common way to enrich a general numpy array, a dask array, or any array like object with the functionality nata adds on top of arrays. It is also the recommended way to create an grid object.

<div class="alert alert-block alert-warning">
    Grid objects can be created by calling <code>GridDataset(...)</code> and instantiating it from the class object directly. This use is discouraged and is recommended for advanced users only.
</div>

## Using array-like objects

Creating a grid from an array-like object can be done by calling the class method `GridDataset.from_array`

In [3]:
grid = GridDataset.from_array([[1, 2, 3], [3, 4, 5]])
grid

name,unnamed
label,unlabeled
unit,''
backend,None
grid_ndim,2
grid_shape,"(2, 3)"
axes,"axis0, axis1"
ndim,2
shape,"(2, 3)"
dtype,int64


## Naming grids, labelling grids, and attaching units to grids

In [4]:
grid = GridDataset.from_array([[1, 2, 3], [3, 4, 5]], name="my_new_name", label="My New Label", unit="some unit")
grid

name,my_new_name
label,My New Label
unit,some unit
backend,None
grid_ndim,2
grid_shape,"(2, 3)"
axes,"axis0, axis1"
ndim,2
shape,"(2, 3)"
dtype,int64


And modified, e.g. for `name`

In [5]:
grid.name = "some_new_name"
print(f"{grid.name = }")

grid.name = 'some_new_name'


In [19]:
try:
    grid = GridDataset.from_array([[1, 2, 3], [3, 4, 5]], name="some invalid name")
except ValueError as exc:
    print(exc, file=sys.stderr)

Argument 'name' has to be an identifier


In [20]:
try:
    grid.name = "some invalid name"
except ValueError as exc:
    print(exc, file=sys.stderr)

New name has to be an identifier


## Axes of grids

Calling this method uses an implicit instantiation. It means that nata uses default values for the common properties, e.g. name, label, axes, to ensure consistancy when dealing with grid objects. This properties can be changed by assigning a new value. Alternatively, a grid can be created by using an explicit instantiation, e.g. passing arguments to the function call.

In [8]:
grid = GridDataset.from_array(
    [[1, 2, 3], [3, 4, 5]],
    time=[1.2, 5.5],
)
grid

name,unnamed
label,unlabeled
unit,''
backend,None
grid_ndim,1
grid_shape,"(3,)"
axes,"time, axis0"
ndim,2
shape,"(2, 3)"
dtype,int64


In [9]:
grid.axes

time,Axis(name: time)
iteration,None
grid_axes,"Axis(name: time), Axis(name: axis0)"


If the argument `time` is provideded, then the first axis for the grid axes correspond to the time axis. Alternatively, an iteration axis can also be provided as another temporal axis.

In [10]:
grid = GridDataset.from_array(
    [[1, 2, 3], [3, 4, 5]],
    iteration=[1, 5],
)
grid

name,unnamed
label,unlabeled
unit,''
backend,None
grid_ndim,1
grid_shape,"(3,)"
axes,"iteration, axis0"
ndim,2
shape,"(2, 3)"
dtype,int64


In [11]:
grid = GridDataset.from_array(
    [[1, 2, 3], [3, 4, 5]],
    iteration=[1, 5],
    time=[2.2, 5.5],
)
grid

name,unnamed
label,unlabeled
unit,''
backend,None
grid_ndim,1
grid_shape,"(3,)"
axes,"time, axis0"
ndim,2
shape,"(2, 3)"
dtype,int64


In [12]:
print(f"{np.array(grid.axes.time) = }")
print(f"{np.array(grid.axes.iteration) = }")

np.array(grid.axes.time) = array([2.2, 5.5])
np.array(grid.axes.iteration) = array([1, 5])


In [13]:
print(f"{grid.axes[0] = }")
print(f"{grid.axes[1] = }")

grid.axes[0] = Axis(name=time)
grid.axes[1] = Axis(name=axis0)


In [14]:
print(f"{grid.axes['time'] = }")
print(f"{grid.axes['axis0'] = }")

grid.axes['time'] = Axis(name=time)
grid.axes['axis0'] = Axis(name=axis0)


In [15]:
grid = GridDataset.from_array(
    [[1, 2, 3], [3, 4, 5]],
    dataset_axes=GridDatasetAxes(
        axes=[
            Axis([1.2, 3.4], name="my_custom_axis0"),
            Axis([-4, 8, 3], name="my_custom_axis1")
        ],
        time=Axis(0.0, name="my_time_axis")
    ),
    # if 'dataset_axes' specified, 'iteration', and 'time' will be ignored
    iteration=[1, 2, 3, 4],
    time=[0],
)
grid.axes

time,Axis(name: my_time_axis)
iteration,None
grid_axes,"Axis(name: my_custom_axis0), Axis(name: my_custom_axis1)"


In [16]:
try:
    grid.axes.iteration = Axis(0, name="my_iteration_axis")
except AttributeError as exc:
    print(exc, file=sys.stderr)

can't set attribute


In [17]:
new_grid = GridDataset.from_array(
    grid,
    dataset_axes=GridDatasetAxes(
        axes=grid.axes.grid_axes,
        iteration=Axis(10, name="my_iteration_axis"),
        time=grid.axes.time,
    )
)

# or equivalent:
new_grid = GridDataset.from_array(
    grid,
    grid_axes=grid.axes.grid_axes,
    iteration=Axis(10, name="my_iteration_axis"),
    time=grid.axes.time,
)
new_grid.axes

time,Axis(name: time)
iteration,Axis(name: iteration)
grid_axes,"Axis(name: my_custom_axis0), Axis(name: my_custom_axis1)"


## Array properties 

<div class="alert alert-block alert-info">
Any reference to an array assumes numpy like arrays.
</div>

Each `GridDataset` acts as it would be an array by providing common array properties. This properties are read-only and can be modified by array functions, e.g., `np.reshape`.

In [18]:
print(f"{grid_ds.shape = }")
print(f"{grid_ds.ndim  = }")
print(f"{grid_ds.dtype = }")

NameError: name 'grid_ds' is not defined

In addition to the array properties, 

Also an extra information about the unit for `GridDataset` and by default it is empty. However, it behaves similar to a numpy array by providing properties like `.shape`, `.ndim` and `.dtype`.



In general, a `GridDataset` is expected to behave like an array.